#Home Work 3
Dicussed and collaborated with : Sushant and Sylvia

References:
1.https://www.youtube.com/watch?v=3gzI4Z2OFgY&t=1172s
2.https://www.youtube.com/watch?v=r6Ijqo5E3I4
3.https://github.com/ShinoharaYuuyoru/NoiseReductionUsingGRU
4.https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU


In [ ]:
#Path: TensorFlow/Tejas/
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.9.2


In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
1.7211919259998467
GPU (s):
0.04042952400004651
GPU speedup over CPU: 42x


##Dependencies

In [ ]:
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import librosa
from IPython.display import Audio

In [ ]:
import numpy as np
import random
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
import keras.utils
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import  Dense, Dropout, Activation, Flatten
from tensorflow.keras import optimizers
from librosa.core import stft,istft
import soundfile as sf
from sklearn.model_selection import train_test_split

import tensorflow as tf
%matplotlib inline

##Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_loc = '/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Audiodata/timit-homework/tr/' # has trx trs and trn in unsorted manner
val_loc = '/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Audiodata/timit-homework/v/'
test_loc = '/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Audiodata/timit-homework/te/'

In [ ]:
tr_list = os.listdir(train_loc)
tr_list.sort() #sorted all files to be in the right order

In [ ]:
print(tr_list)

['trn0000.wav', 'trn0001.wav', 'trn0002.wav', 'trn0003.wav', 'trn0004.wav', 'trn0005.wav', 'trn0006.wav', 'trn0007.wav', 'trn0008.wav', 'trn0009.wav', 'trn0010.wav', 'trn0011.wav', 'trn0012.wav', 'trn0013.wav', 'trn0014.wav', 'trn0015.wav', 'trn0016.wav', 'trn0017.wav', 'trn0018.wav', 'trn0019.wav', 'trn0020.wav', 'trn0021.wav', 'trn0022.wav', 'trn0023.wav', 'trn0024.wav', 'trn0025.wav', 'trn0026.wav', 'trn0027.wav', 'trn0028.wav', 'trn0029.wav', 'trn0030.wav', 'trn0031.wav', 'trn0032.wav', 'trn0033.wav', 'trn0034.wav', 'trn0035.wav', 'trn0036.wav', 'trn0037.wav', 'trn0038.wav', 'trn0039.wav', 'trn0040.wav', 'trn0041.wav', 'trn0042.wav', 'trn0043.wav', 'trn0044.wav', 'trn0045.wav', 'trn0046.wav', 'trn0047.wav', 'trn0048.wav', 'trn0049.wav', 'trn0050.wav', 'trn0051.wav', 'trn0052.wav', 'trn0053.wav', 'trn0054.wav', 'trn0055.wav', 'trn0056.wav', 'trn0057.wav', 'trn0058.wav', 'trn0059.wav', 'trn0060.wav', 'trn0061.wav', 'trn0062.wav', 'trn0063.wav', 'trn0064.wav', 'trn0065.wav', 'trn0066.

##Data Loading

In [ ]:
'''
N_stft = []
X_stft = []
S_stft = []
for files in tr_list:
  #print(files)
  if files.startswith('trn'):
    #print('TRN')
    n, sr = librosa.load('/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Audiodata/timit-homework/tr/'+files, sr=None)
    N = librosa.stft(n, n_fft=1024, hop_length=512)
    N_stft.append(np.abs(N))
  elif files.startswith('trx'):
    #print('TRX')
    x,sr = librosa.load('/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Audiodata/timit-homework/tr/'+files, sr=None)
    X = librosa.stft(x, n_fft=1024, hop_length=512)
    X_stft.append(np.abs(X))
  elif files.startswith('trs'):
    #print('TRS')
    s,sr = librosa.load('/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Audiodata/timit-homework/tr/'+files, sr=None)
    S = librosa.stft(s, n_fft=1024, hop_length=512)
    S_stft.append(np.abs(S))
'''



"\nN_stft = []\nX_stft = []\nS_stft = []\nfor files in tr_list:\n  #print(files)\n  if files.startswith('trn'):\n    #print('TRN')\n    n, sr = librosa.load('/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Audiodata/timit-homework/tr/'+files, sr=None)\n    N = librosa.stft(n, n_fft=1024, hop_length=512)\n    N_stft.append(np.abs(N))\n  elif files.startswith('trx'):\n    #print('TRX')\n    x,sr = librosa.load('/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Audiodata/timit-homework/tr/'+files, sr=None)\n    X = librosa.stft(x, n_fft=1024, hop_length=512)\n    X_stft.append(np.abs(X))\n  elif files.startswith('trs'):\n    #print('TRS')\n    s,sr = librosa.load('/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Audiodata/timit-homework/tr/'+files, sr=None)\n    S = librosa.stft(s, n_fft=1024, hop_length=512)\n    S_stft.append(np.abs(S))\n"

##Functions to Load Train, Test and Validation Data

In [ ]:
def load_data_train(path):
  x_train=[]
  clean_data=[]
  noisy_data=[]
  for file in os.listdir(path):
    if file.startswith('trx') :
      #if file not in x:
        x_train.append(file)

    elif file.startswith('trs') :
      #if file not in clean:
        clean_data.append(file)

    elif file.startswith('trn'):
      #if file not in noise:
        noisy_data.append(file)
  
  x_train.sort()
  clean_data.sort()
  noisy_data.sort()

  return x_train,clean_data,noisy_data

In [ ]:
def load_data_test(path):
  x_test=[]
  clean_data=[]
  noisy_data=[]
  for file in os.listdir(path):
    if file.startswith('tex') :
      #if file not in x:
        x_test.append(file)

    elif  file.startswith('tes') :
      #if file not in clean:
        clean_data.append(file)

    elif  file.startswith('ten') :
      #if file not in noise:
        noisy_data.append(file)
  
  x_test.sort()
  clean_data.sort()
  noisy_data.sort()

  return x_test,clean_data,noisy_data

In [ ]:
def load_data_validation(path):
  x_valid=[]
  clean_data=[]
  noisy_data=[]
  for file in os.listdir(path):
    if file.startswith('vx'):
      #if file not in x:
        x_valid.append(file)

    elif file.startswith('vs'):
      #if file not in clean:
        clean_data.append(file)

    elif file.startswith('vn'):
      #if file not in noise:
        noisy_data.append(file)
  
  x_valid.sort()
  clean_data.sort()
  noisy_data.sort()

  return x_valid,clean_data,noisy_data

##Load Train Test and Validation Data 

In [ ]:
x_train, clean_train, noisy_train = load_data_train(train_loc)

In [ ]:
len(x_train)

1200

In [ ]:
x_train

['trx0000.wav',
 'trx0001.wav',
 'trx0002.wav',
 'trx0003.wav',
 'trx0004.wav',
 'trx0005.wav',
 'trx0006.wav',
 'trx0007.wav',
 'trx0008.wav',
 'trx0009.wav',
 'trx0010.wav',
 'trx0011.wav',
 'trx0012.wav',
 'trx0013.wav',
 'trx0014.wav',
 'trx0015.wav',
 'trx0016.wav',
 'trx0017.wav',
 'trx0018.wav',
 'trx0019.wav',
 'trx0020.wav',
 'trx0021.wav',
 'trx0022.wav',
 'trx0023.wav',
 'trx0024.wav',
 'trx0025.wav',
 'trx0026.wav',
 'trx0027.wav',
 'trx0028.wav',
 'trx0029.wav',
 'trx0030.wav',
 'trx0031.wav',
 'trx0032.wav',
 'trx0033.wav',
 'trx0034.wav',
 'trx0035.wav',
 'trx0036.wav',
 'trx0037.wav',
 'trx0038.wav',
 'trx0039.wav',
 'trx0040.wav',
 'trx0041.wav',
 'trx0042.wav',
 'trx0043.wav',
 'trx0044.wav',
 'trx0045.wav',
 'trx0046.wav',
 'trx0047.wav',
 'trx0048.wav',
 'trx0049.wav',
 'trx0050.wav',
 'trx0051.wav',
 'trx0052.wav',
 'trx0053.wav',
 'trx0054.wav',
 'trx0055.wav',
 'trx0056.wav',
 'trx0057.wav',
 'trx0058.wav',
 'trx0059.wav',
 'trx0060.wav',
 'trx0061.wav',
 'trx006

In [ ]:
len(clean_train)

1200

In [ ]:
len(noisy_train)

1200

In [ ]:
x_validation, clean_validation, noisy_validation = load_data_validation(val_loc)

In [ ]:
len(x_validation)

1200

In [ ]:
len(clean_validation)

1200

In [ ]:
len(noisy_validation)

1200

In [ ]:
x_test, clean_test, noisy_test = load_data_test(test_loc)

##STFT Function ( absolute and transpose taken )

In [ ]:
def stft_calc(path,files):
  data = []
  for i in files:
    s,sr = librosa.load(path+i,sr=None)
    S = librosa.stft(s,n_fft =1024 , hop_length =512)
    pad = np.zeros((513, 178))
    pad[:, :S.shape[1]] = S
    pad=pad.T
    pad_abs = np.abs(pad)
    data.append(pad_abs)
  return data

##STFT for Training Data

In [ ]:
stft_x_train = stft_calc(train_loc,x_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


In [ ]:
stft_clean_train = stft_calc(train_loc,clean_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


In [ ]:
stft_noisy_train = stft_calc(train_loc,noisy_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


In [ ]:
print("Shape of stft_x_train is : ",stft_x_train[0].shape)
print("Shape of stft_clean_train  is : ",stft_clean_train [0].shape)
print("Shape of stft_noisy_train is : ",stft_noisy_train[0].shape)

Shape of stft_x_train is :  (178, 513)
Shape of stft_clean_train  is :  (178, 513)
Shape of stft_noisy_train is :  (178, 513)


##STFT for Validation Data

In [ ]:
stft_x_val = stft_calc(val_loc,x_validation)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


In [ ]:
stft_clean_val = stft_calc(val_loc, clean_validation)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


In [ ]:
stft_noisy_val= stft_calc(val_loc, noisy_validation)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


In [ ]:
print("Shape of stft_x_val is : ",stft_x_val[0].shape)
print("Shape of stft_clean_val  is : ",stft_clean_val[0].shape)
print("Shape of stft_noisy_val is : ",stft_noisy_val[0].shape)

Shape of stft_x_val is :  (178, 513)
Shape of stft_clean_val  is :  (178, 513)
Shape of stft_noisy_val is :  (178, 513)


##IBM Matrix

In [ ]:
train_ibm = [(stft_clean_train[i] > stft_noisy_train[i]).astype(int) for i in range(len(stft_clean_train))]

In [ ]:
print(len(stft_clean_train))
print(len(stft_noisy_train))

1200
1200


In [ ]:
train_ibm = np.stack(train_ibm)
noise_train_new = np.stack(stft_noisy_train)

##IBM Matrix for validation

In [ ]:
val_ibm = [(stft_clean_val[i] > stft_noisy_val[i]).astype(int) for i in range(len(stft_clean_val))]

In [ ]:
val_ibm = np.stack(val_ibm)
noise_val_new = np.stack(stft_noisy_val)

##Model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.GRU(513, return_sequences=True,dropout = 0.2),
    tf.keras.layers.GRU(513, return_sequences=True,dropout = 0.2),
    tf.keras.layers.GRU(513, return_sequences=True),
    tf.keras.layers.Dense(513,activation = 'sigmoid')
])

In [ ]:
model.compile(optimizer = 'adam',loss = 'binary_crossentropy')

In [ ]:
history = model.fit(noise_train_new, train_ibm, epochs=20, batch_size=10)

Epoch 1/20
120/120 [==============================] - 13s 60ms/step - loss: 0.4651
Epoch 2/20
120/120 [==============================] - 7s 59ms/step - loss: 0.3348
Epoch 3/20
120/120 [==============================] - 7s 60ms/step - loss: 0.3276
Epoch 4/20
120/120 [==============================] - 7s 61ms/step - loss: 0.3056
Epoch 5/20
120/120 [==============================] - 7s 61ms/step - loss: 0.2937
Epoch 6/20
120/120 [==============================] - 7s 61ms/step - loss: 0.2926
Epoch 7/20
120/120 [==============================] - 7s 59ms/step - loss: 0.3055
Epoch 8/20
120/120 [==============================] - 7s 59ms/step - loss: 0.2946
Epoch 9/20
120/120 [==============================] - 7s 59ms/step - loss: 0.2878
Epoch 10/20
120/120 [==============================] - 7s 59ms/step - loss: 0.2874
Epoch 11/20
120/120 [==============================] - 7s 59ms/step - loss: 0.2856
Epoch 12/20
120/120 [==============================] - 7s 59ms/step - loss: 0.2829
Epoch 13/20


In [ ]:
model.save('/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Models/GRUmodel.h5')

In [ ]:
model.evaluate(noise_val_new, val_ibm, batch_size=10)

120/120 [==============================] - 4s 27ms/step - loss: 0.3007


0.3006991446018219

###SNR Function

In [ ]:
def snr_calc(clean, reconstruct):
    print(clean.shape)
    recon = reconstruct[:clean.shape[0]]
    return 10*np.log10(np.sum(clean**2)/np.sum(clean - reconstruct)**2)

##Predict IBM Mask

In [ ]:
predmodel = model.predict(noise_val_new)

38/38 [==============================] - 3s 39ms/step


In [ ]:
def stft_calc_2(path,input):
  data_abs=[]
  data=[]
  for i in input:
    s, sr = librosa.load(path+i, sr=None)
    S = librosa.stft(s, n_fft=1024, hop_length=512)
    padding = np.zeros((513, 178))
    padding[:, :S.shape[1]] = S
    padding=padding.T
    pad_abs = np.abs(padding)
    data.append(padding)
    data_abs.append(pad_abs)
  return data,data_abs

In [ ]:
def stft_calc_3(path,input):
  data_abs=[]
  data=[]
  for i in input:
    s, sr = librosa.load(path+i, sr=None)
    S = librosa.stft(s, n_fft=1024, hop_length=512)
    padding = np.zeros((513, 178))
    padding[:, :S.shape[1]] = S
    padding=padding.T
    pad_abs = np.abs(padding)
    data.append(S)
    data_abs.append(pad_abs)
  return data,data_abs

In [ ]:
val_data_noisy,noisy_val_2 = stft_calc_2(val_loc,noisy_validation)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [ ]:
val_data_clean,clean_val_2 = stft_calc_3(val_loc,clean_validation)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [ ]:
val_data_noisy = np.stack(val_data_noisy)

##Recover Validation Sequence

In [ ]:
predclean = np.multiply(predmodel, val_data_noisy)

##SNR Calc !

In [ ]:
snr_values = []
for i in range(len(val_data_clean)):
    St = librosa.istft(val_data_clean[i], hop_length=512)
    St2 = librosa.istft(predclean[i].T, hop_length=512)
    s = sum((St)**2)
    St2 = St2[:St.shape[0]]
    #diff_st = sum(np.subtract(S_test,S_test_2)**2)+1e-20
    SNR = 10*np.log10(np.sum(St**2)/np.sum(St - St2)**2)
    snr_values.append(SNR)

In [ ]:
snr_values

[16.38892795071706,
 15.615083560995306,
 14.665395473238991,
 36.93242456059883,
 18.297234053259924,
 18.59006302751704,
 15.479006744525023,
 20.156330637872557,
 18.601987474305428,
 16.137552297591288,
 15.007599385456864,
 19.735694702208097,
 29.6108047403142,
 19.37216910570872,
 30.236119749786194,
 16.63748944643444,
 14.158572638585639,
 18.906308134004338,
 11.763260534141688,
 14.558349344700359,
 27.20540068563401,
 21.487152216235653,
 33.96537094076403,
 18.361116455623158,
 19.621504279266876,
 19.55182817059688,
 18.144369936265846,
 18.617527283321575,
 29.823031845068925,
 21.90555873533297,
 22.78771237893333,
 23.586969143045554,
 35.88999963497824,
 25.20061685923001,
 27.692583138111267,
 24.337327522013187,
 20.118929373990163,
 23.913184910062334,
 26.916495093574426,
 29.1175033243263,
 23.101941005000274,
 22.003931749203502,
 21.186107368132134,
 19.772027227982637,
 16.91259549894182,
 20.698285660207617,
 26.82393266625438,
 20.05897718951855,
 23.8648370

In [ ]:
import statistics
statistics.mean(snr_values)

20.531155842283795

##Test Data Set

In [ ]:
def stft_calc_4(path,input):
  data_abs=[]
  data=[]
  for i in input:
    s, sr = librosa.load(path+i, sr=None)
    S = librosa.stft(s, n_fft=1024, hop_length=512)
    padding = np.zeros((513, 195))
    padding[:, :S.shape[1]] = S
    padding=padding.T
    pad_abs = np.abs(padding)
    data.append(padding)
    data_abs.append(pad_abs)
  return data,data_abs

In [ ]:
test_noisy_data,noise_test_new = stft_calc_4(test_loc,x_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [ ]:
noisy_test_2 = np.stack(noise_test_new)

##IBM Mask for test Set

In [ ]:
test_predmodel =  model.predict(noisy_test_2, batch_size=10)

40/40 [==============================] - 2s 29ms/step


In [ ]:
noisy_test_data = np.stack(test_noisy_data)

##Generate Clean sequence from test

In [ ]:
test_cleanpred = np.multiply(test_predmodel, noisy_test_data)

In [ ]:
store = '/content/drive/MyDrive/ENGR-E-533 /DATA/HW3/Output/'

for i in range(len(x_test)):
  test_s , test_sr = librosa.load(test_loc+x_test[i], sr=None)
  test_out = librosa.istft(test_cleanpred[i].T, win_length=1024, hop_length=512)
  test_out = test_out[:test_s.shape[0]]
  sf.write(store+x_test[i]+"_cleaned.wav",test_out,test_sr)

In [ ]:
%%shell
jupyter nbconvert --to html /content/Deep_Learning_HW3_Q3.ipynb

[NbConvertApp] Converting notebook /content/Deep_Learning_HW3_Q3.ipynb to html
[NbConvertApp] Writing 490393 bytes to /content/Deep_Learning_HW3_Q3.html
